# Utilization Prediction: Prediction of hourly utilization of the two sites

## Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
os.environ["KERAS_BACKEND"] = "torch"

import keras
from keras import Sequential
from keras.layers import Dense

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%matplotlib inline

## Load and prepare data

In [2]:
# Load data
charging_data = pd.read_pickle("data/charging_modified.pkl")
weather_data = pd.read_csv("data/weather_modified.csv")

# drop not important columns in weather data
# TODO: what is this unnamed column here
weather_data.drop(columns = ["city", "cloud_cover_description", "pressure", "felt_temperature", "Unnamed: 0"], inplace = True)
weather_data["timestamp"] = pd.to_datetime(weather_data["timestamp"], utc = True).dt.floor(freq = "H")

# change columns into value per hour for every session
charging_data["kWhPerHour"] = charging_data["kWhDelivered"] / (charging_data["ChargingTimeHours"] + charging_data["NoChargingTimeHours"])
charging_data["user_minutesAvailablePerHour"] = charging_data["user_minutesAvailable"] / (charging_data["ChargingTimeHours"] + charging_data["NoChargingTimeHours"])
charging_data["user_milesRequestedPerHour"] = charging_data["user_milesRequested"] / (charging_data["ChargingTimeHours"] + charging_data["NoChargingTimeHours"])
charging_data["user_kWhRequestedPerHour"] = charging_data["user_kWhRequested"] / (charging_data["ChargingTimeHours"] + charging_data["NoChargingTimeHours"])

# drop not important columns in charging data
charging_data.drop(columns = ["kWhDelivered", "kWhPerMinute", "NoChargingTime", "user_modifiedAt", "doneChargingTime", "NoChargingTimeMinutes", "userRegistered", 
                              "spaceID", "stationID", "ChargingTime", "user_paymentRequired_values", "user_requestedDeparture", "userID", "ChargingTimeMinutes",
                              "user_kWhRequestFulfilment", "user_kWhRequested", "user_milesRequested", "user_minutesAvailable", "month", "weekday"], inplace = True)

# Delete rows with null values
charging_data = charging_data.dropna()

# delete duplicate row index
charging_data.reset_index(drop = True, inplace = True)

charging_data = charging_data.astype({
    "kWhPerHour": "float",
    "NoChargingTimeHours": "float",
    "ChargingTimeHours": "float",
    "user_kWhRequestedPerHour": "float",
    "user_minutesAvailablePerHour": "float",
    "user_milesRequestedPerHour": "float",
    "user_WhPerMile": "int64",
    "siteID": "int64"
})

weather_data.astype({
    "temperature": "float",
    "cloud_cover": "float",
    "windspeed": "float",
    "precipitation": "float"
})

charging_data

,id,connectionTime,disconnectTime,siteID,NoChargingTimeHours,ChargingTimeHours,user_WhPerMile,kWhPerHour,user_minutesAvailablePerHour,user_milesRequestedPerHour,user_kWhRequestedPerHour
0,5bc915caf9af8b0dad3c0660,2018-04-30 15:07:49+00:00,2018-05-01 00:27:55+00:00,2,0.001111,9.333889,350,5.121371,58.918050,18.211034,6.373862
1,5bc917d0f9af8b0dc677b8bb,2018-05-07 14:38:18+00:00,2018-05-08 01:15:01+00:00,2,0.002222,10.609722,400,2.608664,5.654006,1.884669,0.753867
2,5bc9190ff9af8b0dc677b9c1,2018-05-11 15:17:01+00:00,2018-05-12 00:18:43+00:00,2,1.213056,7.815278,350,1.936681,60.476278,5.538121,1.938342
3,5bc919c3f9af8b0dc677ba32,2018-05-14 13:50:26+00:00,2018-05-14 23:09:45+00:00,2,2.434722,6.887222,400,1.265294,6.436425,2.145475,0.858190
4,5bc919c3f9af8b0dc677ba62,2018-05-15 00:06:39+00:00,2018-05-15 01:13:58+00:00,2,0.582500,0.539444,250,2.741669,544.590245,17.826195,4.456549
...,...,...,...,...,...,...,...,...,...,...,...
35385,6002419ff9af8b50514f4c45,2020-12-30 20:49:50+00:00,2020-12-30 21:25:49+00:00,1,0.003333,0.596389,279,4.096897,500.231589,133.395090,37.217230
35386,6002419cf9af8b50514f4c3f,2020-12-30 23:02:11+00:00,2020-12-31 02:53:06+00:00,2,2.737778,1.110833,290,0.583067,31.180079,5.196680,1.507037
35387,6002419ff9af8b50514f4c46,2020-12-31 03:56:09+00:00,2020-12-31 06:04:33+00:00,1,0.471944,1.668056,370,2.368692,56.074766,14.018692,5.186916
35388,6003931bf9af8b5261c8c794,2020-12-31 16:45:14+00:00,2020-12-31 17:30:59+00:00,2,0.001389,0.761111,283,6.936393,629.508197,131.147541,37.114754


## Create feature vector X and labels Y

In [3]:
# split time interval between connectionTime and disconnectTime in minutes for every hour
def split_time_interval(index, start_date, end_date):
    indices = pd.DatetimeIndex([start_date])
    indices = indices.append(pd.date_range(start_date.ceil(freq = "H"), end_date.floor(freq = "H"), freq = "H"))
    indices = indices.append(pd.DatetimeIndex([end_date]))

    time_list = list(map(lambda x : 60 if x == 0 else x, indices.minute))
    time_list[0] = 60 - time_list[0]
    
    return list((time_list, indices))


# creates table with session id, charging time in minutes per hour and timestamps
def create_minute_table(id, minute_array, siteID):
    return list(zip([id] * len(minute_array[0]), minute_array[0], minute_array[1], [siteID] * len(minute_array[0])))

# groups entires by timestamp
def group_by_timestamp(X):
    X = X.groupby("timestamp").agg({"kWhPerHour": "sum",
                                     "NoChargingTimeHours": "sum",
                                     "ChargingTimeHours": "sum",
                                     "siteID": "mean",
                                     "user_kWhRequestedPerHour": "sum",
                                     "user_minutesAvailablePerHour": "sum",
                                     "user_milesRequestedPerHour": "sum",
                                     "user_WhPerMile": "mean"
                               })
    return X

In [4]:
# concat single minute lists to one big list with lists of tuples containing id, charging times in minutes per hour and timestamps
temp = []

for index in charging_data.index :
    charging_times = split_time_interval(charging_data.index[index], 
                                         charging_data.loc[index, "connectionTime"], 
                                         charging_data.loc[index, "disconnectTime"])
    temp.append(create_minute_table(charging_data.loc[index, "id"], charging_times, charging_data.loc[index, "siteID"]))
    
charging_data.drop("siteID", inplace = True, axis = 1)

# parse list of lists into four lists: ids, charging time in minutes per hour, timestamps and siteIDs
ids, charging_times, timestamps, siteIDs = map(list, zip(*[tuple for list in temp for tuple in list]))

# create data.frame with four columns: session id, charging time, starting timestamp in minutes for every hour and siteID
charging_time_per_hour = pd.DataFrame(
                            {"id": ids,
                             "charging_time_per_hour": charging_times,
                             "timestamp": timestamps,
                             "siteID": siteIDs})

charging_time_per_hour.loc[:, "timestamp"] = charging_time_per_hour.loc[:, "timestamp"].dt.floor(freq = "H")

### Labels Y:

In [19]:
# roughly 50 EV charging stations per site * 60 minutes
maximum_utilization = 50 * 60

# divide data set into two sets: one for each site
Y1 = charging_time_per_hour.loc[charging_time_per_hour["siteID"] == 1]
Y2 = charging_time_per_hour.loc[charging_time_per_hour["siteID"] == 2]

# create labels for utilization rate by diving the hourly utilization in minutes by the maximum utilization
Y1 = pd.Series(map(lambda x : x / maximum_utilization, Y1.groupby("timestamp").sum().loc[:, "charging_time_per_hour"]))
Y2 = pd.Series(map(lambda x : x / maximum_utilization, Y2.groupby("timestamp").sum().loc[:, "charging_time_per_hour"]))

# combine the sets together into one
Y = pd.concat([Y1, Y2], axis = 0)
charging_time_per_hour

,id,charging_time_per_hour,timestamp,siteID
0,5bc915caf9af8b0dad3c0660,53,2018-04-30 15:00:00+00:00,2
1,5bc915caf9af8b0dad3c0660,60,2018-04-30 16:00:00+00:00,2
2,5bc915caf9af8b0dad3c0660,60,2018-04-30 17:00:00+00:00,2
3,5bc915caf9af8b0dad3c0660,60,2018-04-30 18:00:00+00:00,2
4,5bc915caf9af8b0dad3c0660,60,2018-04-30 19:00:00+00:00,2
...,...,...,...,...
324377,6003931cf9af8b5261c8c79d,51,2020-12-31 18:00:00+00:00,1
324378,6003931cf9af8b5261c8c79d,60,2020-12-31 19:00:00+00:00,1
324379,6003931cf9af8b5261c8c79d,60,2020-12-31 20:00:00+00:00,1
324380,6003931cf9af8b5261c8c79d,60,2020-12-31 21:00:00+00:00,1


### Feature Vector X:

In [6]:
# join charge time per hour with the rest of the charging data
X_temp = charging_time_per_hour.merge(charging_data, how = "left", on = "id").drop(["connectionTime", "disconnectTime", "id", "charging_time_per_hour"], axis = 1)

# divide data set into two sets: one for each site
X1 = X_temp.loc[X_temp["siteID"] == 1]
X2 = X_temp.loc[X_temp["siteID"] == 2]

# group columns by timestamps
X1 = group_by_timestamp(X1)
X2 = group_by_timestamp(X2)

# combine both subsets together into one
X = pd.concat([X1, X2], axis = 0)
X = X.astype({"siteID": "int64"})

# add weather data to feature vector
X = X.merge(weather_data, how = "left", on = "timestamp").drop_duplicates(["timestamp", "siteID"]).fillna(0)

# add weekday and month as categorical variable
X["weekday"] = X["timestamp"].dt.day_name()
X["month"] = X["timestamp"].dt.month_name()
X = X.drop("timestamp", axis = 1)

# create dummies for categorical variables
X = pd.get_dummies(X, columns = ["siteID"])
X = pd.get_dummies(X, columns = ["weekday"])
X = pd.get_dummies(X, columns = ["month"])
X

,kWhPerHour,NoChargingTimeHours,ChargingTimeHours,user_kWhRequestedPerHour,user_minutesAvailablePerHour,user_milesRequestedPerHour,user_WhPerMile,temperature,cloud_cover,windspeed,...,month_December,month_February,month_January,month_July,month_June,month_March,month_May,month_November,month_October,month_September
0,5.062274,1.288611,31.019444,7.806032,113.471696,27.066743,319.0,13.0,34.0,0.0,...,False,False,False,False,False,False,False,False,True,False
1,24.681760,11.918889,145.693611,35.781542,649.251995,103.372592,359.8,14.0,28.0,0.0,...,False,False,False,False,False,False,False,False,True,False
4,24.681760,11.918889,145.693611,35.781542,649.251995,103.372592,359.8,17.0,30.0,0.0,...,False,False,False,False,False,False,False,False,True,False
5,24.681760,11.918889,145.693611,35.781542,649.251995,103.372592,359.8,18.0,34.0,6.0,...,False,False,False,False,False,False,False,False,True,False
6,24.681760,11.918889,145.693611,35.781542,649.251995,103.372592,359.8,20.0,34.0,7.0,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26514,0.583067,2.737778,1.110833,1.507037,31.180079,5.196680,290.0,17.0,34.0,0.0,...,True,False,False,False,False,False,False,False,False,False
26515,0.583067,2.737778,1.110833,1.507037,31.180079,5.196680,290.0,15.0,33.0,6.0,...,True,False,False,False,False,False,False,False,False,False
26516,1.166135,5.475556,2.221667,3.014074,62.360159,10.393360,290.0,13.0,33.0,0.0,...,True,False,False,False,False,False,False,False,False,False
26517,6.936393,0.001389,0.761111,37.114754,629.508197,131.147541,283.0,9.0,30.0,6.0,...,True,False,False,False,False,False,False,False,False,False


## Cross-validation: split data in training and test set

In [7]:
# Split data into training set and testing set and normalize
scaler = StandardScaler()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.3, random_state = 30)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

pandas.core.series.Series

## Prediction model 1: Neural Network

In [8]:
# Build the neural network
network = Sequential()

# Add layers
network.add(Dense(input_shape = (32,), units = 15, activation = "relu"))
network.add(Dense(units = 15, activation = "relu"))
network.add(Dense(units = 1, activation = "sigmoid"))

In [9]:
# Compiling the neural network
network.compile(optimizer = "adam", 
                   loss = "mean_squared_error", 
                   metrics = ["mean_squared_error"])
network.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Layer (type)                       ┃ Output Shape                  ┃     Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ dense (Dense)                      │ (None, 15)                    │         495 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dense_1 (Dense)                    │ (None, 15)                    │         240 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dense_2 (Dense)                    │ (None, 1)                     │          16 │
└────────────────────────────────────┴───────────────────────────────┴─────────────┘

 Total params: 751 (2.93 KB)

 Trainable params: 751 (2.93 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Fitting the Neural Network
network.fit(X_train, Y_train, batch_size=50, epochs=100)

Epoch 1/100
331/331 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0665 - mean_squared_error: 0.0665
Epoch 2/100
331/331 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0118 - mean_squared_error: 0.0118
Epoch 3/100
331/331 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0090 - mean_squared_error: 0.0090
Epoch 4/100
331/331 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0079 - mean_squared_error: 0.0079
Epoch 5/100
331/331 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0077 - mean_squared_error: 0.0077
Epoch 6/100
331/331 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0078 - mean_squared_error: 0.0078
Epoch 7/100
331/331 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0070 - mean_squared_error: 0.0070
Epoch 8/100
331/331 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0075 - mean_squared_error: 0.0075
Epoch 9/100
331/331 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0065 - mean_squared_error: 0.0065
Epoch 10/100
331/331 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0070 - mean_squared_error: 0.0070
Epoch 11/100
331/331 ━━━━━━━━

### Performance evaluation

In [11]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score

mse_nn  = mean_squared_error(Y_test, network.predict(X_test))
mae_nn  = mean_absolute_error(Y_test, network.predict(X_test))
msle_nn = mean_squared_log_error(Y_test, network.predict(X_test))
print(mse_nn)
print(mae_nn)
print(msle_nn)

222/222 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
222/222 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
222/222 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
0.006936195983030264
0.03027455472232551
0.0016292298380031


## Prediction Model 2: Decision Tree

In [12]:
from sklearn.tree import DecisionTreeRegressor, export_graphviz
import graphviz

# fit decision tree for regression
reg_tree = DecisionTreeRegressor(max_depth = 10, criterion = "squared_error")
reg_tree.fit(X_train, Y_train)

DecisionTreeRegressor(max_depth=10)

In [13]:
# Plot tree
data = export_graphviz(reg_tree, feature_names = X.columns)
graph = graphviz.Source(data)

graph

### Performance Evaluation

In [14]:
# Evaluate performance
mse_dt    = mean_squared_error(Y_test, reg_tree.predict(X_test))
mae_dt    = mean_absolute_error(Y_test, reg_tree.predict(X_test))
r2_dt     = r2_score(Y_test, reg_tree.predict(X_test))
print(mse_dt)
print(mae_dt)
print(r2_dt)

0.0022217970307730475
0.025545200039351155
0.9799713953815244
